In [8]:
import numpy as np
import pandas as pd

In [14]:
df = pd.read_parquet('data/itub_2021.parquet')

In [15]:
divs_df = pd.read_parquet('data/itub_divs_2021.parquet')

In [16]:
df = df.join(divs_df)
df['has_div'] = df['value'].map(lambda v: not np.isnan(v))

In [19]:
df[df['has_div']]

,open,close,next_open,overnight,div_type,value,has_div
2021-01-22,29.25,29.27,28.99,-0.009566,JCP,0.050160,True
2021-01-29,29.22,28.34,29.00,0.023289,Dividendo,0.015000,True
2021-02-25,26.90,26.08,26.20,0.004601,Dividendo,0.096407,True
2021-02-25,26.90,26.08,26.20,0.004601,JCP,0.050580,True
2021-02-26,26.20,25.55,25.90,0.013699,Dividendo,0.015000,True
2021-03-25,27.00,27.65,27.52,-0.004702,JCP,0.050640,True
2021-03-31,28.31,27.97,28.13,0.005720,Dividendo,0.015000,True
2021-04-27,27.47,27.09,27.52,0.015873,JCP,0.056480,True
2021-04-30,27.18,27.53,27.70,0.006175,Dividendo,0.015000,True
2021-05-24,29.52,29.35,29.32,-0.001022,JCP,0.048740,True


I can already see this won't work because many of the days with divs actually have positive returns, which shouldn't be the case. I can maybe make it work for identifying very large divs, I'll try vale later, for now I'll just run it to the end and see the results

Intuition:<br>
event = there was a dividend<br>
known = overnight return < -0.01<br>

P(event | known) = P(known | event) * P(event) / P(known)

Using -0.01 as a threshold for now, I'll try to optimize this later

Obs.: I'll be calling both JCP and Dividendo as dividend from here on, for this test it's fine

In [26]:
threshold = -0.01
prob_div = len(df[df['has_div']]) / len(df)
prob_neg_ret = len(df[df['overnight'] < threshold]) / len(df)
prob_neg_ret_by_div = len(df[(df['overnight'] < threshold) & (df['has_div'])]) / \
                      len(df[df['has_div']])

print(f'probability there was a dividend = {100 * prob_div:.3}%')
print(f'probability of overnight return < -0.01 = {100 * prob_neg_ret:.3}%')
print(f'probability of overnight return < -0.01 when there\'s a dividend = {100 * prob_neg_ret_by_div:.3}%')

probability there was a dividend = 7.69%
probability of overnight return < -0.01 = 10.1%
probability of overnight return < -0.01 when there's a dividend = 5.26%


probability of overnight return < -0.01 is itself larger than when there's a div, which already defeats the premise

In [27]:
prob = prob_div * prob_neg_ret / prob_neg_ret_by_div
print(f'probability there was a dividend given the overnight return was < -0.01 = {100 * prob:.3}%')

probability there was a dividend given the overnight return was < -0.01 = 14.8%
